In [3]:
import torch
from torch import nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## the deep learning model start


1. data into tensors
2. pytorch model
3. make predictions
4. evaluate the model
5. improve through experimentation
6. save and reload the model

### !. Data ( getting and cleaning the data )

* can have structued , unstructured data , semi strucutred data




In [4]:
#creating data

weight = 0.7
bias = 0.5


start = 0
end = 1
step =0.03
X = torch.arange(start,end,step).unsqueeze(dim=1)
Y = weight * X +bias

X[:10],Y[:10]

(tensor([[0.0000],
         [0.0300],
         [0.0600],
         [0.0900],
         [0.1200],
         [0.1500],
         [0.1800],
         [0.2100],
         [0.2400],
         [0.2700]]),
 tensor([[0.5000],
         [0.5210],
         [0.5420],
         [0.5630],
         [0.5840],
         [0.6050],
         [0.6260],
         [0.6470],
         [0.6680],
         [0.6890]]))

In [5]:
###divding the data into train and test set

In [ ]:
train_split = int(0.8* len(X))
X_train,Y_train = X[:train_split],y[:train_split]
X_test,Y_test = X[train_split:],Y[train_split]

len(X-train),len(y_train)